In [1]:
!pip install --upgrade scikit-learn

In [2]:
import sklearn

print("scikit-learn version:", sklearn.__version__)

scikit-learn version: 1.3.0


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d ebrahimhaquebhatti/pakistan-house-price-prediction

pakistan-house-price-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/pakistan-house-price-prediction.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [7]:
import numpy as np
data=pd.read_csv("Entities.csv")

In [8]:
data.isnull().sum()

Unnamed: 0           0
property_id          0
location_id          0
page_url             0
property_type        0
price                0
location             0
city                 0
province_name        0
latitude             0
longitude            0
baths                0
purpose              0
bedrooms             0
date_added           0
agency           44071
agent            44072
Total_Area           0
dtype: int64

In [9]:
data.shape

(168446, 18)

In [10]:
data.head()

,Unnamed: 0,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,purpose,bedrooms,date_added,agency,agent,Total_Area
0,0,237062,3325,https://www.zameen.com/Property/g_10_g_10_2_gr...,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,For Sale,2,2/4/2019,NaN,NaN,1089.004
1,1,346905,3236,https://www.zameen.com/Property/e_11_2_service...,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,For Sale,3,5/4/2019,NaN,NaN,15246.056
2,2,386513,764,https://www.zameen.com/Property/islamabad_g_15...,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,For Sale,5,7/17/2019,NaN,NaN,2178.008
3,3,656161,340,https://www.zameen.com/Property/islamabad_bani...,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,For Sale,4,4/5/2019,NaN,NaN,10890.000
4,4,841645,3226,https://www.zameen.com/Property/dha_valley_dha...,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,For Sale,3,7/10/2019,Easy Property,Muhammad Junaid Ceo Muhammad Shahid Director,2178.008


In [11]:
columns_to_drop = ['Unnamed: 0','location_id','page_url','date_added','agent','agency','latitude','longitude','province_name']
data = data.drop(columns=columns_to_drop)

In [12]:
data.head()

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area
0,237062,Flat,10000000,G-10,Islamabad,2,For Sale,2,1089.004
1,346905,Flat,6900000,E-11,Islamabad,3,For Sale,3,15246.056
2,386513,House,16500000,G-15,Islamabad,6,For Sale,5,2178.008
3,656161,House,43500000,Bani Gala,Islamabad,4,For Sale,4,10890.000
4,841645,House,7000000,DHA Defence,Islamabad,3,For Sale,3,2178.008


In [13]:
data['property_id'].duplicated().sum()

0

In [14]:
data.describe()

,property_id,price,baths,bedrooms,Total_Area
count,1.684460e+05,1.684460e+05,168446.000000,168446.000000,1.684460e+05
mean,1.559626e+07,1.776576e+07,2.874227,3.179422,1.394239e+04
std,2.251207e+06,3.531003e+07,2.463400,1.971401,8.623647e+05
min,8.657500e+04,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,1.488320e+07,1.750000e+05,0.000000,2.000000,1.905757e+03
50%,1.665851e+07,8.500000e+06,3.000000,3.000000,4.356016e+03
75%,1.708662e+07,1.950000e+07,4.000000,4.000000,1.197904e+04
max,1.735772e+07,2.000000e+09,403.000000,68.000000,3.387988e+08


In [15]:
data[data.baths==0].shape

(42382, 9)

In [16]:
data['Total_Area']=data['Total_Area'].astype(int)

In [17]:
df2 = data[data['Total_Area'] > 100000].index
data.drop(df2, inplace=True)
data.reset_index(drop=True, inplace=True)  # Reset index and drop the old index column
print(data.shape)

(167295, 9)


In [18]:
data.tail()

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area
167290,17355248,House,26500000,Gadap Town,Karachi,0,For Sale,6,26136
167291,17355249,House,12500000,Gadap Town,Karachi,0,For Sale,3,2178
167292,17355250,House,27000000,Gadap Town,Karachi,0,For Sale,6,26136
167293,17355251,House,11000000,Gadap Town,Karachi,0,For Sale,3,21235
167294,17355287,House,9000000,Bahria Town Karachi,Karachi,3,For Sale,3,25591


In [19]:
df=data.loc[(data['bedrooms']==0) | (data['baths']==0)].index
df

Int64Index([    18,     43,     45,     50,     51,     52,     55,     56,
                78,     79,
            ...
            167243, 167244, 167245, 167251, 167254, 167289, 167290, 167291,
            167292, 167293],
           dtype='int64', length=42502)

In [20]:
# Calculate the mean of non-zero bedrooms
mean_bedrooms = data.loc[data["bedrooms"] > 0, "bedrooms"].mean().astype(int)
mean_bedrooms

3

In [21]:
# Replace 0 bedrooms with the mean
data.loc[data["bedrooms"] == 0, "bedrooms"] = mean_bedrooms

In [22]:
# Calculate the mean of non-zero bathrooms
mean_baths = data.loc[data["baths"] > 0, "baths"].mean().astype(int)
mean_baths

3

In [23]:
# Replace 0 bathrooms with the mean
data.loc[data["baths"] == 0, "baths"] = mean_baths

In [24]:
data.loc[18]

property_id                 2384979
property_type                  Flat
price                       1750000
location         PWD Housing Scheme
city                      Islamabad
baths                             3
purpose                    For Sale
bedrooms                          3
Total_Area                     4083
Name: 18, dtype: object

In [25]:
df=data.loc[(data['bedrooms']==0) | (data['baths']==0)].index
df.shape

(0,)

In [26]:
data['price_per_sqfeet']=data['price']/data['Total_Area']

In [27]:
data.describe()

,property_id,price,baths,bedrooms,Total_Area,price_per_sqfeet
count,1.672950e+05,1.672950e+05,167295.000000,167295.000000,167295.000000,1.672950e+05
mean,1.559733e+07,1.714223e+07,3.618829,3.572522,9409.835709,inf
std,2.249533e+06,3.196652e+07,1.841051,1.513868,13735.863303,NaN
min,8.657500e+04,0.000000e+00,1.000000,1.000000,0.000000,0.000000e+00
25%,1.488409e+07,1.700000e+05,3.000000,3.000000,1905.000000,3.431373e+01
50%,1.665996e+07,8.500000e+06,3.000000,3.000000,4356.000000,1.215959e+03
75%,1.708662e+07,1.940000e+07,4.000000,4.000000,11434.000000,7.347539e+03
max,1.735772e+07,1.430000e+09,403.000000,28.000000,98010.000000,inf


In [28]:
data['location'].value_counts()

DHA Defence                21944
Bahria Town Karachi         8524
Bahria Town Rawalpindi      8477
Bahria Town                 6404
Gulistan-e-Jauhar           5862
                           ...  
Dilawar Colony                 1
Lawyers Housing Society        1
Victoria Heights               1
Jamal Homes                    1
Sundar                         1
Name: location, Length: 1502, dtype: int64

In [29]:
data.describe()

,property_id,price,baths,bedrooms,Total_Area,price_per_sqfeet
count,1.672950e+05,1.672950e+05,167295.000000,167295.000000,167295.000000,1.672950e+05
mean,1.559733e+07,1.714223e+07,3.618829,3.572522,9409.835709,inf
std,2.249533e+06,3.196652e+07,1.841051,1.513868,13735.863303,NaN
min,8.657500e+04,0.000000e+00,1.000000,1.000000,0.000000,0.000000e+00
25%,1.488409e+07,1.700000e+05,3.000000,3.000000,1905.000000,3.431373e+01
50%,1.665996e+07,8.500000e+06,3.000000,3.000000,4356.000000,1.215959e+03
75%,1.708662e+07,1.940000e+07,4.000000,4.000000,11434.000000,7.347539e+03
max,1.735772e+07,1.430000e+09,403.000000,28.000000,98010.000000,inf


In [30]:
(data['Total_Area']/data['bedrooms']).describe()

count    167295.000000
mean       2928.862204
std        4043.416463
min           0.000000
25%         544.400000
50%        1089.000000
75%        4356.000000
max       98010.000000
dtype: float64

In [31]:
data=data[((data['Total_Area']/data['bedrooms'])>=300)]
data.describe()

,property_id,price,baths,bedrooms,Total_Area,price_per_sqfeet
count,1.566770e+05,1.566770e+05,156677.000000,156677.000000,156677.000000,156677.000000
mean,1.560549e+07,1.776605e+07,3.621157,3.546577,9983.115926,3896.295844
std,2.246338e+06,3.285835e+07,1.571070,1.501196,14009.725817,5189.671244
min,8.657500e+04,0.000000e+00,1.000000,1.000000,544.000000,0.000000
25%,1.491028e+07,1.400000e+05,3.000000,3.000000,2178.000000,28.650138
50%,1.666286e+07,9.000000e+06,3.000000,3.000000,5445.000000,1010.101010
75%,1.708732e+07,2.000000e+07,4.000000,4.000000,11979.000000,7086.614173
max,1.735772e+07,1.430000e+09,14.000000,28.000000,98010.000000,316804.407713


In [32]:
def  remove_outlier_sqft(df):
    df_output=pd.DataFrame()
    for key,subdf in data.groupby('location'):
        m=np.mean(subdf.price_per_sqfeet)
        st=np.std(subdf.price_per_sqfeet)
        gen_df=subdf[(subdf.price_per_sqfeet>(m-st))&(subdf.price_per_sqfeet <= (m+st))]
        df_output=pd.concat( [df_output , gen_df ], ignore_index=True)
    return df_output

data=remove_outlier_sqft(data)
data.describe()

,property_id,price,baths,bedrooms,Total_Area,price_per_sqfeet
count,1.237070e+05,1.237070e+05,123707.00000,123707.000000,123707.000000,123707.000000
mean,1.563804e+07,1.387707e+07,3.55085,3.466530,11049.585577,2753.901020
std,2.225483e+06,2.287092e+07,1.53442,1.465798,14753.547707,3546.616583
min,3.469050e+05,0.000000e+00,1.00000,1.000000,544.000000,0.000000
25%,1.497308e+07,8.500000e+04,3.00000,3.000000,2178.000000,22.038567
50%,1.668884e+07,7.000000e+06,3.00000,3.000000,5445.000000,717.401286
75%,1.709042e+07,1.700000e+07,4.00000,4.000000,13068.000000,5693.296602
max,1.735772e+07,6.000000e+08,14.00000,28.000000,98010.000000,36737.692873


In [33]:
df1=data.loc[(data['bedrooms']>=20) | (data['baths']>=15)].index
df1.shape

(8,)

In [34]:
data.describe()

,property_id,price,baths,bedrooms,Total_Area,price_per_sqfeet
count,1.237070e+05,1.237070e+05,123707.00000,123707.000000,123707.000000,123707.000000
mean,1.563804e+07,1.387707e+07,3.55085,3.466530,11049.585577,2753.901020
std,2.225483e+06,2.287092e+07,1.53442,1.465798,14753.547707,3546.616583
min,3.469050e+05,0.000000e+00,1.00000,1.000000,544.000000,0.000000
25%,1.497308e+07,8.500000e+04,3.00000,3.000000,2178.000000,22.038567
50%,1.668884e+07,7.000000e+06,3.00000,3.000000,5445.000000,717.401286
75%,1.709042e+07,1.700000e+07,4.00000,4.000000,13068.000000,5693.296602
max,1.735772e+07,6.000000e+08,14.00000,28.000000,98010.000000,36737.692873


In [35]:
status_counts = data['purpose'].value_counts()

print("Counts of properties for sale and for rent:")
print(status_counts)

Counts of properties for sale and for rent:
For Sale    81357
For Rent    42350
Name: purpose, dtype: int64


In [36]:
data.isnull().sum()

property_id         0
property_type       0
price               0
location            0
city                0
baths               0
purpose             0
bedrooms            0
Total_Area          0
price_per_sqfeet    0
dtype: int64

In [37]:
data['Total_Area'].dtype

dtype('int64')

In [38]:
data['property_type'].unique()

array(['Upper Portion', 'Penthouse', 'House', 'Flat', 'Room',
       'Lower Portion', 'Farm House'], dtype=object)

In [39]:
data.head()

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,price_per_sqfeet
0,9154510,Upper Portion,40000,12th Avenue,Islamabad,3,For Rent,3,3811,10.495933
1,9154511,Upper Portion,40000,12th Avenue,Islamabad,3,For Rent,3,3811,10.495933
2,15022622,Penthouse,40000,12th Avenue,Islamabad,4,For Rent,4,2722,14.695077
3,12038300,House,7000000,204 Chak Road,Faisalabad,4,For Sale,3,8984,779.162956
4,12178174,House,6200000,204 Chak Road,Faisalabad,3,For Sale,2,1361,4555.473916


In [40]:
data['purpose'].unique()

array(['For Rent', 'For Sale'], dtype=object)

In [41]:
column_name =[ 'property_type','location','city']

In [42]:
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
data['property_type'] = data['property_type'].str.lower()
data['location'] = data['location'].str.lower()
data['city'] = data['city'].str.lower()

In [44]:
data['location'] = data['location'].apply(lambda x: ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(x)]))
data['city'] = data['city'].apply(lambda x: ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(x)]))

In [45]:
data.head()

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,price_per_sqfeet
0,9154510,upper portion,40000,12th avenu,islamabad,3,For Rent,3,3811,10.495933
1,9154511,upper portion,40000,12th avenu,islamabad,3,For Rent,3,3811,10.495933
2,15022622,penthouse,40000,12th avenu,islamabad,4,For Rent,4,2722,14.695077
3,12038300,house,7000000,204 chak road,faisalabad,4,For Sale,3,8984,779.162956
4,12178174,house,6200000,204 chak road,faisalabad,3,For Sale,2,1361,4555.473916


In [46]:
data['location'] = data['location'] + ' , ' + data['city']
data.drop(['price_per_sqfeet'], axis=1, inplace=True)
data.head()

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area
0,9154510,upper portion,40000,"12th avenu , islamabad",islamabad,3,For Rent,3,3811
1,9154511,upper portion,40000,"12th avenu , islamabad",islamabad,3,For Rent,3,3811
2,15022622,penthouse,40000,"12th avenu , islamabad",islamabad,4,For Rent,4,2722
3,12038300,house,7000000,"204 chak road , faisalabad",faisalabad,4,For Sale,3,8984
4,12178174,house,6200000,"204 chak road , faisalabad",faisalabad,3,For Sale,2,1361


In [47]:
data['city'].unique()

array(['islamabad', 'faisalabad', 'karachi', 'lahor', 'rawalpindi'],
      dtype=object)

In [48]:
postcode_lookup = {
    'islamabad': '44000',
    'faisalabad': '38000',
    'karachi':'75000',
    'lahor':'54000',
    'rawalpindi':'46000'

}
data['postcode'] = data['city'].map(postcode_lookup)
data.head()

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,postcode
0,9154510,upper portion,40000,"12th avenu , islamabad",islamabad,3,For Rent,3,3811,44000
1,9154511,upper portion,40000,"12th avenu , islamabad",islamabad,3,For Rent,3,3811,44000
2,15022622,penthouse,40000,"12th avenu , islamabad",islamabad,4,For Rent,4,2722,44000
3,12038300,house,7000000,"204 chak road , faisalabad",faisalabad,4,For Sale,3,8984,38000
4,12178174,house,6200000,"204 chak road , faisalabad",faisalabad,3,For Sale,2,1361,38000


In [49]:
valid_property_types = ["flat", "house"]
sale_data = data[data['property_type'].isin(valid_property_types)]
data = data.sample(frac=1, random_state=42)

In [50]:
# Separate data for sale and for rent
sale_data = data[data['purpose'] == 'For Sale']
rent_data = data[data['purpose'] == 'For Rent']

In [51]:
rent_data.shape

(42350, 10)

In [52]:
data.tail()

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,postcode
119879,14567378,house,23000000,"state life hous societi , lahor",lahor,5,For Sale,4,2722,54000
103694,15320634,house,17500000,"north karachi , karachi",karachi,8,For Sale,6,13068,75000
860,17085775,house,6000000,"adiala road , rawalpindi",rawalpindi,3,For Sale,2,1361,46000
15795,13987050,flat,20000,"bahria town rawalpindi , rawalpindi",rawalpindi,2,For Rent,1,9801,46000
121958,16908347,flat,5900000,"univers road , karachi",karachi,3,For Sale,3,11434,75000


In [53]:
rent_data = rent_data[rent_data['price'] >= 5000]
rent_data = rent_data[rent_data['price'] <= 80000]
sale_data=sale_data[sale_data['price']>=100000]

In [54]:
rent_data.shape

(30343, 10)

In [55]:
# Delete the last  rows from the DataFrame
sale_data = sale_data.iloc[:-60000]
rent_data=rent_data.iloc[:-20000]
sale_data = sale_data.reset_index(drop=True)
rent_data = rent_data.reset_index(drop=True)

In [56]:
sale_data.shape
rent_data.shape

(10343, 10)

In [57]:
# text vectorization
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [58]:
sale_vectors=cv.fit_transform(sale_data['location']).toarray()
rent_vectors=cv.fit_transform(rent_data['location']).toarray()

In [59]:
properties_in_location = sale_data[sale_data['location'] == 'gadap town , karachi']
properties_in_location

,property_id,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,postcode
118,17321987,flat,720000,"gadap town , karachi",karachi,2,For Sale,2,7623,75000
134,17320210,flat,2000000,"gadap town , karachi",karachi,2,For Sale,2,5989,75000
197,16717042,house,1300000,"gadap town , karachi",karachi,3,For Sale,3,7078,75000
204,14849936,house,7500000,"gadap town , karachi",karachi,3,For Sale,3,13068,75000
229,17173167,house,11500000,"gadap town , karachi",karachi,3,For Sale,3,13068,75000
...,...,...,...,...,...,...,...,...,...,...
21047,13764727,house,1500000,"gadap town , karachi",karachi,2,For Sale,2,6534,75000
21061,16644539,flat,1900000,"gadap town , karachi",karachi,1,For Sale,1,4356,75000
21099,17246317,flat,1900000,"gadap town , karachi",karachi,1,For Sale,1,4356,75000
21176,16981896,house,9600000,"gadap town , karachi",karachi,3,For Sale,3,17424,75000


In [60]:
from sklearn.metrics.pairwise import cosine_similarity
input1=input("what you want: 'for sale' or 'for rent' :  ")

what you want: 'for sale' or 'for rent' :  for sale


In [61]:
def recommend(location,user_postcode):
    if input1 == 'for sale':
        matching_rows = sale_data[sale_data['postcode'] == user_postcode]
        cities = ['karachi', 'lahor', 'islamabad', 'faisalabad', 'rawalpindi']
        if location in cities:
            properties_in_location = matching_rows[matching_rows['city'] == location]
        else:
            properties_in_location = matching_rows[matching_rows['location'] == location]

        if properties_in_location.empty:
            print("No matching properties found.")
            return

        low_price = properties_in_location['price'].min()
        max_price = properties_in_location['price'].max()
        estimated_price = properties_in_location['price'].mean()


        return low_price, estimated_price, max_price

    elif input1 == 'for rent':
        cities = ['karachi', 'lahor', 'islamabad', 'faisalabad', 'rawalpindi']
        matching_rows = rent_data[rent_data['postcode'] == user_postcode]
        if location in cities:
            properties_in_location = matching_rows[matching_rows['city'] == location]
        else:
            properties_in_location = matching_rows[matching_rows['location'] == location]

        if properties_in_location.empty:
            print("No matching properties found.")
            return

        low_price = properties_in_location['price'].min()
        max_price = properties_in_location['price'].max()
        estimated_price = properties_in_location['price'].mean()


        return low_price, estimated_price, max_price

In [62]:
user_postcode = input("Enter a postcode: ")
location=input("Enter your Location : ")
stemmed_input = ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(location)])

Enter a postcode: 75000
Enter your Location : karachi


In [63]:
stemmed_input

'karachi'

In [64]:
result=recommend(stemmed_input,user_postcode)

In [65]:
if result:
    low_price, estimated_price, max_price = result
    print(f"Low Price: {low_price}")
    print(f"Estimated Price: {estimated_price}")
    print(f"Max Price: {max_price}")
else:
    print("No matching properties found.")

Low Price: 502000
Estimated Price: 17643940.903430317
Max Price: 390000000


In [66]:
# Save the trained model to a pickle file
import pickle
with open("property_model.pkl", "wb") as file:
    pickle.dump(recommend, file)

In [67]:
pickle.dump(sale_data.to_dict(),open('sale_dict1.pkl','wb'))
pickle.dump(rent_data.to_dict(),open('rent_dict1.pkl','wb'))